# Algorytmy tekstowe - laboraorium 2

# 1. Zadanie polega na implementacji dwóch algorytmów kompresji:

### 1. statycznego algorytmu Huffmana (2 p)

In [119]:
from queue import PriorityQueue
from collections import Counter

class Node:
    def __init__(self, frequency, sign=None):
        self.frequency = frequency
        self.sign = sign
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.frequency < other.frequency


def build_tree_static(signs):
    frequencies = Counter(signs)
    nodes = [Node(frequencies[sign], sign) for sign in frequencies]
    pq = PriorityQueue()
    root = None
    
    for node in nodes:
        pq.put(node)
    
    while True:
        left = pq.get()
        right = pq.get()
        parent = Node(left.frequency + right.frequency)
        parent.left, parent.right = left, right
        pq.put(parent)
        
        if pq.qsize() == 1: # root
            root = pq.get()
            break

    return root


def build_code_table(node, codes = {}, code = ""):
    if node.sign is not None:
        codes[node.sign] = code
        
    if node.left is not None:
        build_code_table(node.left, codes, code + "0")
    if node.right is not None:
        build_code_table(node.right, codes, code + "1")

    return codes

def static_huffman_encode(data):
    root = build_tree_static(data)
    codes = build_code_table(root)
    
    result = ""
    
    for sign in data:
        result += codes[sign]
        
    return result, root

def static_huffman_decode(encoded_data, root):
    decoded_data = ""
    node = root

    for bit in encoded_data:
        if bit == "1": node = node.right
        else: node = node.left

        if node.sign is not None:
            decoded_data += node.sign
            node = root

    return decoded_data

Test działnia:

In [120]:
data = "abracadabra"
result, root = static_huffman_encode(data)
print("Encoded data: " + result)
print("Is implemented correctly?: " + str(data == static_huffman_decode(result, root)))

Encoded data: 01111100100010101111100
Is implemented correctly?: True


### 2. dynamicznego algorytmu Huffmana (3 p)

In [125]:
class Node:
    def __init__(self, sign=None, weight=0, parent=None, left=None, right=None):
        self.sign = sign
        self.weight = weight
        self.parent = parent
        self.left = left
        self.right = right
        
    def __lt__(self, other):
        return self.weight < other.weight

def update_tree(node): # przechodzenie po drzewie metoda bottom-up i naprawianie drzewa
    parent = node.parent
    
    if parent is None: # mamy korzeń - koniec rekurencji, nie trzeba już nic poprawiać
        return
    
    nodes = [parent.left, parent.right]
    
    if nodes[0].weight > nodes[1].weight or nodes[0] is not node:
        nodes[0], nodes[1] = nodes[1], nodes[0]
        
    update_tree(parent)

def get_code(node):
    code = []
    while node.parent is not None:
        if node == node.parent.left:
            code.append('0')
        else:
            code.append('1')
        node = node.parent
    return list(reversed(code))

def build_tree_dynamic(data): # tworzenie drzewa Huffmana, łączy dwa najmniejsze węzły w kazdym kroku 
    sign_nodes = {}
    
    for sign in data: # inicjalizacja każdego znaku jako Node w slowniku node'ów
        sign_nodes[sign] = sign_nodes.get(sign, Node(sign, 0))
        sign_nodes[sign].weight += 1

    leaves = list(sign_nodes.values())
    
    while len(leaves) > 1:
        leaves.sort(key=lambda n: (n.weight, id(n)))
        left, right = leaves.pop(0), leaves.pop(0)
        parent = Node(None, left.weight + right.weight, None, left, right)
        left.parent = right.parent = parent
        leaves.append(parent)
    
    return leaves[0], sign_nodes

## kodowanie i dekodowanie
def dynamic_huffman_encode(data):
    root, sign_nodes = build_tree_dynamic(data)
    bits = []
    for sign in data:
        bits.extend(get_code(sign_nodes[sign]))
    return ''.join(bits), root

def dynamic_huffman_decode(bits, root):
    current = root
    signs = []
    for bit in bits:
        if bit == '0':
            current = current.left
        else:
            current = current.right
        if current.sign is not None:
            signs.append(current.sign)
            current = root
    return ''.join(signs)

Test działania:

In [126]:
data = "abracadabra"
result, root = dynamic_huffman_encode(data)
print("Encoded data: " + result)
print("Is implemented correctly?: " + str(data == dynamic_huffman_decode(result, root)))

Encoded data: 01111001100011010111100
Is implemented correctly?: True


# Dla każdego z algorytmów należy wykonać następujące zadania:

#### Opracować format pliku przechowującego dane. Zwróć uwagę na dwie kwestie:

<ol>
    <li> Liczba bitów wynikowego pliku nie musi być podzielna przez 8, ale z dysku zawsze odczytujemy pełne bajty, dlatego ważne jest, aby jakoś rozwiązać ten problem. W przeciwnym razie po dekompresji można uzyskać nadmiarowe dane. </li>
    <li> Plik wynikowy musi być binarny, tzn. rozwiązanie nie może zakładać, że w pliku tym zapisywane są 0 i 1 jako znaki ASCII. </li>
</ol>

#### Pomysł:

Na początku pliku zapisywana jest wartość paddingu, która określa liczbę bitów na końcu zakodowanych danych, które należy zignorować przy dekompresji (aby dlugosc tekstu byla podzielna przez 8 dodano na koniec w brakujące miejsca 0). Następnie każde 8 bitów zakodowanych danych są zapisywane jako pojedynczy bajt w pliku binarnym. 

# Zaimplementować algorytm kompresji i dekompresji danych dla tego formatu pliku.

#### Huffman statyczny:

In [121]:
def static_huffman_encode_file(data, output_file):
    root = build_tree_static(data)
    codes = build_code_table(root)
    
    encoded_data = ""
    
    for sign in data:
        encoded_data += codes[sign]
    
    # Obliczam, ile zer jest potrzebnych na końcu, aby uzupełnić ostatni bajt.
    padding = 8 - (len(encoded_data) % 8)
    encoded_data += padding * "0"
    

    with open(output_file, "wb") as file:
        # na początku pliku jest wpisana ilosc zer do zignorowania
        file.write(bytes([padding]))
        
        # wpisywanie zakodowanych znakow do pliku
        for i in range(0, len(encoded_data), 8):
            byte = encoded_data[i:i+8]
            file.write(bytes([int(byte, 2)]))
    
    return root


def static_huffman_decode_file(input_file, root):

    with open(input_file, "rb") as file:
        # wczytanie ilosci zer do zignorowania
        padding = file.read(1)[0]
        
        # wczytywanie kodow z pliku 
        encoded_data = ""
        byte = file.read(1)
        while byte:
            encoded_data += bin(byte[0])[2:].zfill(8)
            byte = file.read(1)
    
    # usuwanie dodanych zer z konca
    encoded_data = encoded_data[:-padding]
    
    decoded_data = ""
    node = root
    
    for bit in encoded_data:
        if bit == "1": node = node.right
        else: node = node.left

        if node.sign is not None:
            decoded_data += node.sign
            node = root

    return decoded_data

Testowanie:

In [122]:
data = "aabbbcdddde"
encoded_file = "encoded.bin"
decoded_file = "decoded.txt"

# kodowanie
root = static_huffman_encode_file(data, encoded_file)

# dekodowanie
decoded_data = static_huffman_decode_file(encoded_file, root)

with open(decoded_file, "w") as file:
    file.write(decoded_data)

# sprawdzenie poprawności
with open(decoded_file, "r") as file:
    if file.read() == data:
        print("Implemented correctly")

Implemented correctly


#### Huffman dynamiczny:

In [130]:
def dynamic_huffman_encode_file(data, output_file):
    root, codes = build_tree_dynamic(data)
    
    encoded_data = ""
    
    for sign in data:
        encoded_data += codes[sign].sign
        
    bits = []
    for sign in encoded_data:
        bits.extend(get_code(codes[sign]))
        
    encoded_data = ''.join(bits)
    
    # Obliczam, ile zer jest potrzebnych na końcu, aby uzupełnić ostatni bajt.
    padding = 8 - (len(encoded_data) % 8)
    encoded_data += padding * "0"
    

    with open(output_file, "wb") as file:
        # na początku pliku jest wpisana ilosc zer do zignorowania
        file.write(bytes([padding]))
        
        # wpisywanie zakodowanych znakow do pliku
        for i in range(0, len(encoded_data), 8):
            byte = encoded_data[i:i+8]
            file.write(bytes([int(byte, 2)]))
    
    return root

def dynamic_huffman_decode_file(input_file, root):

    with open(input_file, "rb") as file:
        # wczytanie ilosci zer do zignorowania
        padding = file.read(1)[0]
        
        # wczytywanie kodow z pliku 
        encoded_data = ""
        byte = file.read(1)
        while byte:
            encoded_data += bin(byte[0])[2:].zfill(8)
            byte = file.read(1)
    
    # usuwanie dodanych zer z konca
    encoded_data = encoded_data[:-padding]
    
    decoded_data = ""
    node = root
    
    for bit in encoded_data:
        if bit == "1": node = node.right
        else: node = node.left

        if node.sign is not None:
            decoded_data += node.sign
            node = root

    return decoded_data

Testowanie:

In [132]:
data = "aabbbcdddde"
encoded_file = "encoded1.bin"
decoded_file = "decoded1.txt"

# kodowanie
root = dynamic_huffman_encode_file(data, encoded_file)

# dekodowanie
decoded_data = dynamic_huffman_decode_file(encoded_file, root)
with open(decoded_file, "w") as file:
    file.write(decoded_data)

# sprawdzenie poprawności
with open(decoded_file, "r") as file:
    if file.read() == data:
        print("Implemented correctly")

Implemented correctly


# Zmierzyć współczynnik kompresji (wyrażone w procentach: 1 - plik_skompresowany / plik_nieskompresowany) dla plików o rozmiarach: 1kB, 10kB, 100kB, 1MB, o różnej zawartości:

<ol>
    <li>wybrany przez Ciebie plik tekstowy z projektu <a href="https://www.gutenberg.org">Gutenberg</a>,</li>
    <li>wybrany przez Ciebie plik z kodem źródłowym <a href="https://github.com/torvalds/linux">jądra Linuksa</a>, </li>
    <li>plik ze znakami losowanymi z rozkładu jednostajnego - należy uwzględnić wszystkie 256 wartości, a nie tylko znaki drukowalne. </li>
</ol>

# W sumie w punkcie 3 należy przeprowadzić analizę dla łącznie 12 plików (4 rozmiary x 3 typy plików).

# Zmierzyć czas kompresji i dekompresji dla plików z punktu 3.

Pomocnicze funkcje:

Podpunkt 1:

Podpunkt 2

Podpunkt 3: